In [ ]:
# Original model parameters but training data set reduced to 10% volume
import tensorflow as tf
from tensorflow.keras import datasets, models, layers, callbacks
import numpy as np

# Reduced training data: 6,000 samples (randomly sampled from the original training set)
# Load and preprocess CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# Reduce training set size
reduced_train_images = train_images[:6000]
reduced_train_labels = train_labels[:6000]

# Define baseline parameters
baseline_params = {
    "epochs": 2,
    "learning_rate": 0.001,
    "filters_layer1": 32,
    "filters_layer2": 64,
    "dense_neurons": 64,
    "dropout": None,
    "batch_size": 64
}

# Store results in a dictionary
results = {}

# Baseline Model
print("Running Baseline Model...")
baseline_model = models.Sequential()
baseline_model.add(layers.Conv2D(baseline_params["filters_layer1"], (3, 3), activation='relu', input_shape=(32, 32, 3)))
baseline_model.add(layers.MaxPooling2D((2, 2)))
baseline_model.add(layers.Conv2D(baseline_params["filters_layer2"], (3, 3), activation='relu'))
baseline_model.add(layers.MaxPooling2D((2, 2)))
baseline_model.add(layers.Conv2D(baseline_params["filters_layer2"], (3, 3), activation='relu'))
baseline_model.add(layers.Flatten())
baseline_model.add(layers.Dense(baseline_params["dense_neurons"], activation='relu'))
if baseline_params["dropout"]:
    baseline_model.add(layers.Dropout(baseline_params["dropout"]))
baseline_model.add(layers.Dense(10))

# Compile baseline model
baseline_optimizer = tf.keras.optimizers.Adam(learning_rate=baseline_params["learning_rate"])
baseline_model.compile(optimizer=baseline_optimizer,
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])

# Early stopping callback
early_stopping = callbacks.EarlyStopping(
    monitor='val_accuracy', patience=3, restore_best_weights=True
)

# Train baseline model
baseline_history = baseline_model.fit(
    reduced_train_images,
    reduced_train_labels,
    epochs=baseline_params["epochs"],
    batch_size=baseline_params["batch_size"],
    validation_data=(test_images, test_labels),
    callbacks=[early_stopping],
    verbose=0
)

# Evaluate baseline model
baseline_test_loss, baseline_test_acc = baseline_model.evaluate(test_images, test_labels, verbose=0)

# Store baseline results
results["baseline"] = {
    "parameters": baseline_params,
    "test_accuracy": baseline_test_acc
}
print(f"Baseline Test Accuracy: {baseline_test_acc:.4f}")

Running Baseline Model...


c:\Users\Marinand\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Baseline Test Accuracy: 0.4191


In [ ]:
# Code block to loop through different parameter combinations and store results from model performance
import tensorflow as tf
from tensorflow.keras import datasets, models, layers, callbacks
import numpy as np

# Reduced training data: 6,000 samples (randomly sampled from the original training set)
# Load and preprocess CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# Reduce training set size
reduced_train_images = train_images[:6000]
reduced_train_labels = train_labels[:6000]

# Parameter combinations to test
epochs_list = [5]
learning_rate_list = [0.001, 0.0001]
filters_layer1_list = [32, 64]
filters_layer2_list = [64, 128]
dense_neurons_list = [64, 128]
dropout_list = [None, 0.3]
batch_size_list = [32, 64]

# Generate all parameter combinations
from itertools import product
param_combinations = list(product(epochs_list, learning_rate_list, filters_layer1_list,
                                   filters_layer2_list, dense_neurons_list, dropout_list, batch_size_list))

# Early stopping callback
early_stopping = callbacks.EarlyStopping(
    monitor='val_accuracy', patience=3, restore_best_weights=True
)

# Initialize results dictionary to store the results
results = {}

# Loop through parameter combinations
for i, params in enumerate(param_combinations):
    epochs, learning_rate, filters_layer1, filters_layer2, dense_neurons, dropout, batch_size = params
    print(f"Testing combination {i + 1}/{len(param_combinations)}: {params}")

    # Build model
    model = models.Sequential()
    model.add(layers.Conv2D(filters_layer1, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters_layer2, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(filters_layer2, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(dense_neurons, activation='relu'))
    if dropout:
        model.add(layers.Dropout(dropout))
    model.add(layers.Dense(10))

    # Compile model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    # Train model
    history = model.fit(
        reduced_train_images,
        reduced_train_labels,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(test_images, test_labels),
        callbacks=[early_stopping],
        verbose=0
    )

    # Evaluate model
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)

    # Store results
    results[i] = {
        "parameters": {
            "epochs": epochs,
            "learning_rate": learning_rate,
            "filters_layer1": filters_layer1,
            "filters_layer2": filters_layer2,
            "dense_neurons": dense_neurons,
            "dropout": dropout,
            "batch_size": batch_size
        },
        "test_accuracy": test_acc
    }

In [ ]:
# Sort results from loop above by test_accuracy in descending order
sorted_results = sorted(results.items(), key=lambda item: item[1]['test_accuracy'], reverse=True)

# Print sorted results as a table
print("\nFinal Results (Sorted by Test Accuracy):")
print("Combination\tTest Accuracy\tParameters")
for key, value in sorted_results:
    params = value['parameters']
    formatted_params = "\n\t\t\t".join([f"{k}: {v}" for k, v in params.items()])
    print(f"{key}\t\t{value['test_accuracy']:.4f}\n\t\t\t{formatted_params}")


Final Results (Sorted by Test Accuracy):
Combination	Test Accuracy	Parameters
28		0.5402
			epochs: 5
			learning_rate: 0.001
			filters_layer1: 64
			filters_layer2: 128
			dense_neurons: 128
			dropout: None
			batch_size: 32
0		0.4810
			epochs: 5
			learning_rate: 0.001
			filters_layer1: 32
			filters_layer2: 64
			dense_neurons: 64
			dropout: None
			batch_size: 32
4		0.3662
			epochs: 5
			learning_rate: 0.001
			filters_layer1: 32
			filters_layer2: 64
			dense_neurons: 128
			dropout: None
			batch_size: 32
20		0.3654
			epochs: 5
			learning_rate: 0.001
			filters_layer1: 64
			filters_layer2: 64
			dense_neurons: 128
			dropout: None
			batch_size: 32
8		0.3650
			epochs: 5
			learning_rate: 0.001
			filters_layer1: 32
			filters_layer2: 128
			dense_neurons: 64
			dropout: None
			batch_size: 32
29		0.3537
			epochs: 5
			learning_rate: 0.001
			filters_layer1: 64
			filters_layer2: 128
			dense_neurons: 128
			dropout: None
			batch_size: 64
24		0.3504
			epochs: 5
			le

In [1]:
# Model implementation using the best performing parameters
# Number of epochs increased to 20

import tensorflow as tf
from tensorflow.keras import datasets, models, layers, callbacks
import numpy as np

# Reduced training data: 6,000 samples (randomly sampled from the original training set)
# Load and preprocess CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# Reduce training set size
reduced_train_images = train_images[:6000]
reduced_train_labels = train_labels[:6000]

# Define the parameters you want to use
best_params = {
    "epochs": 20,
    "learning_rate": 0.001,
    "filters_layer1": 64,
    "filters_layer2": 128,
    "dense_neurons": 128,
    "dropout": None,
    "batch_size": 32
}

# Build model with the chosen parameters
print("Running model with best parameters...")

model = models.Sequential()
model.add(layers.Conv2D(best_params["filters_layer1"], (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(best_params["filters_layer2"], (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(best_params["filters_layer2"], (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(best_params["dense_neurons"], activation='relu'))
if best_params["dropout"]:
    model.add(layers.Dropout(best_params["dropout"]))
model.add(layers.Dense(10))

# Compile model
optimizer = tf.keras.optimizers.Adam(learning_rate=best_params["learning_rate"])
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Early stopping callback
early_stopping = callbacks.EarlyStopping(
    monitor='val_accuracy', patience=3, restore_best_weights=True
)

# Train model
history = model.fit(
    reduced_train_images,
    reduced_train_labels,
    epochs=best_params["epochs"],
    batch_size=best_params["batch_size"],
    validation_data=(test_images, test_labels),
    callbacks=[early_stopping],
    verbose=1  # Change to 1 or 2 to see progress
)

# Evaluate model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

# Print the result
print(f"Test Accuracy: {test_acc:.4f}")

Running model with best parameters...


c:\Users\Marinand\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.1970 - loss: 2.1366 - val_accuracy: 0.3333 - val_loss: 1.8523
Epoch 2/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.3617 - loss: 1.7130 - val_accuracy: 0.4098 - val_loss: 1.6191
Epoch 3/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.4486 - loss: 1.5153 - val_accuracy: 0.4807 - val_loss: 1.4353
Epoch 4/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.5251 - loss: 1.3325 - val_accuracy: 0.4984 - val_loss: 1.3995
Epoch 5/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 10s 51ms/step - accuracy: 0.5614 - loss: 1.2432 - val_accuracy: 0.5072 - val_loss: 1.3598
Epoch 6/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - accuracy: 0.6013 - loss: 1.1071 - val_accuracy: 0.5317 - val_loss: 1.3460
Epoch 7/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 13s 71ms/step - accuracy: 0.6469 - loss: 1.0166 - val_accuracy: 0.5436 - val_loss: 1.3051
Epoch 8/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 14s 74ms/step - accuracy: 0.6799 - loss: 0.9227 - val